In [1]:
import os
import nbimporter

In [2]:
import utils

Importing Jupyter notebook from utils.ipynb


### DIRECTORY PREFIX

In [3]:
prefix = 'release/'

### CREATE A FRESH DIRECTORY

In [4]:
!rm -rf { prefix }

In [5]:
!mkdir { prefix }

### CONVERT NOTEBOOKS TO SCRIPTS

In [6]:
files = os.listdir()

In [7]:
# LOOP THROUGH FILES
for file in files:
    suffix = file[-5:]
    
    # IF THE FILE IS A NOTEBOOK, CONVERT
    if suffix == 'ipynb':
        !jupyter nbconvert --output-dir={ prefix } --to script { file }

[NbConvertApp] Converting notebook utils.ipynb to script
[NbConvertApp] Writing 3918 bytes to release/utils.py
[NbConvertApp] Converting notebook compiler.ipynb to script
[NbConvertApp] Writing 2065 bytes to release/compiler.py
[NbConvertApp] Converting notebook patcher.ipynb to script
[NbConvertApp] Writing 1437 bytes to release/patcher.py
[NbConvertApp] Converting notebook device.ipynb to script
[NbConvertApp] Writing 962 bytes to release/device.py
[NbConvertApp] Converting notebook blockchain.ipynb to script
[NbConvertApp] Writing 2740 bytes to release/blockchain.py
[NbConvertApp] Converting notebook launcher.ipynb to script
[NbConvertApp] Writing 6862 bytes to release/launcher.py
[NbConvertApp] Converting notebook init.ipynb to script
[NbConvertApp] Writing 1602 bytes to release/init.py
[NbConvertApp] Converting notebook oracle.ipynb to script
[NbConvertApp] Writing 170 bytes to release/oracle.py


### COMPILE THE LAUNCHER & PATCHER

In [8]:
!pyinstaller --distpath={ prefix + 'dist' } --workpath={ prefix + 'build' } --specpath={ prefix } --log-level ERROR --onefile { prefix + 'launcher.py' }

In [9]:
!pyinstaller --distpath={ prefix + 'dist' } --workpath={ prefix + 'build' } --specpath={ prefix } --log-level ERROR --onefile { prefix + 'patcher.py' }

### MOVE THE CUSTOM ORACLE & RESOURCES DIR

In [10]:
!cp -r { 'resources' } { prefix + 'dist/resources' }

In [11]:
!mkdir { prefix + 'dist/oracle' }

In [12]:
!mv { prefix + 'oracle.py' } { prefix + 'dist/oracle/main.py' }

### NUKE GARBAGE & EXTRACT THE COMPILED FILES

In [13]:
!mv { prefix + 'dist/' } .

In [14]:
!rm -rf { prefix }

In [15]:
!mv dist/ { prefix }

### CREATE CHECKSUM OBJECT

In [16]:
checksums = {}

In [17]:
files = ['launcher', 'patcher']

In [18]:
for file in files:
    
    # GENERATE CHECKSUM FOR FILE & PUSH
    checksums[file] = utils.generate_checksum(prefix + file)

### SAVE CHECKSUM FILE

In [19]:
utils.save_json(checksums, prefix + 'checksums.json')

### CREATE ZIP ARCHIVE

In [20]:
zip_files = utils.gather_files(prefix)

In [21]:
utils.create_zip(zip_files, prefix, 'middleware.zip')